In [2]:
import torch
from transformers import XLMRobertaForTokenClassification

from transformers import XLMRobertaModel, XLMRobertaTokenizer, AutoTokenizer, AutoModel
tokenizer = XLMRobertaTokenizer.from_pretrained("xlm-roberta-large")
# tokenizer = AutoTokenizer.from_pretrained("intfloat/multilingual-e5-large")

model = AutoModel.from_pretrained("intfloat/multilingual-e5-large-instruct")

In [3]:
import pandas as pd
import csv
from collections import Counter
from tqdm import tqdm, trange

In [ ]:
df_ru = pd.read_csv('corpus/rus-ru_web-public_2019_1M/rus-ru_web-public_2019_1M-sentences.txt', sep='\t', header=None, quoting=csv.QUOTE_NONE)
df_ru.columns = ['idx', 'text']
cnt_ru = Counter()
for text in tqdm(df_ru.text):
    cnt_ru.update(tokenizer.encode(text.lower()))
print(len(cnt_ru), len(cnt_ru)/tokenizer.vocab_size)  

 28%|██▊       | 281461/1000000 [00:16<00:41, 17258.15it/s]

In [ ]:
df_kz = pd.read_csv('corpus/kaz_newscrawl_2016_1M/kaz_newscrawl_2016_1M-sentences.txt', sep='\t', header=None, quoting=csv.QUOTE_NONE)
df_kz.columns = ['idx', 'text']
cnt_kz = Counter()
for text in tqdm(df_kz.text):
    cnt_kz.update(tokenizer.encode(text.lower()))
print(len(cnt_kz), len(cnt_kz)/tokenizer.vocab_size)  

In [ ]:
new_tokens = set(range(1000))

In [ ]:
len(cnt_kz)

In [ ]:
new_tokens = set(range(1000))

for i, (k, v) in enumerate(cnt_ru.most_common(30000)):
    if len(new_tokens) == 25000:
        print(i, 'Russan tokens are included')
        break
    if k not in new_tokens:
        new_tokens.add(k)
for i, (k, v) in enumerate(cnt_kz.most_common(40000)):
    if len(new_tokens) == 30000:
        print(i, 'Kazakh tokens are included')
        break
    if k not in new_tokens:
        new_tokens.add(k)
for t in range(tokenizer.vocab_size - 100, tokenizer.vocab_size):
    new_tokens.add(t)
print(len(new_tokens))
kept_ids = sorted(new_tokens)

In [ ]:
len(kept_ids)

In [ ]:
new_size = len(kept_ids)

new_emb = torch.nn.Embedding(new_size, model.embeddings.word_embeddings.embedding_dim)
for new_id, old_id in enumerate(kept_ids):
    new_emb.weight.data[new_id] = model.embeddings.word_embeddings.weight.data[old_id]

model.embeddings.word_embeddings.weight = new_emb.weight

model.config.vocab_size = new_size
model.config._name_or_path = 'models/bcc'

In [ ]:
new_size

In [ ]:
import sentencepiece.sentencepiece_model_pb2 as spmp
smp = tokenizer.sp_model.serialized_model_proto()
m = spmp.ModelProto()
m.ParseFromString(smp)
print('the loaded model has pieces:', len(m.pieces))
new_pieces = []
for idx in kept_ids[:-2]:
    new_pieces.append(m.pieces[idx])
                     
print('the new pieces:', len(new_pieces))

for i, p in enumerate(new_pieces):
    m.pieces[i].piece = p.piece
    m.pieces[i].score = p.score
    m.pieces[i].type = p.type
    
n = len(new_pieces)
for i in trange(len(m.pieces) - n):
    m.pieces.pop(len(m.pieces) - 1)
    
print(len(m.pieces))
with open('e5-large-instruct-dropped.model', 'wb') as f:
    f.write(m.SerializeToString())
new_tokenizer = XLMRobertaTokenizer('e5-large-instruct-dropped.model', extra_ids=0)

In [ ]:
new_tokenizer.save_pretrained('models/e5-large-instruct-dropped')
model.save_pretrained('models/e5-large-instruct-dropped')

In [ ]:
model = AutoModel.from_pretrained('intfloat/multilingual-e5-large')


In [23]:
import torch.nn.functional as F

from torch import Tensor
from transformers import AutoTokenizer, AutoModel


def average_pool(last_hidden_states: Tensor,
                 attention_mask: Tensor) -> Tensor:
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]


# Each input text should start with "query: " or "passage: ", even for non-English texts.
# For tasks other than retrieval, you can simply use the "query: " prefix.
input_texts = ['query: Менчя',
               'query: 南瓜的家常做法',
               "passage: As a general guideline, the CDC's average requirement of protein for women ages 19 to 70 is 46 grams per day. But, as you can see from this chart, you'll need to increase that if you're expecting or training for a marathon. Check out the chart below to see how much protein you should be eating each day.",
               "passage: 1.清炒南瓜丝 原料:嫩南瓜半个 调料:葱、盐、白糖、鸡精 做法: 1、南瓜用刀薄薄的削去表面一层皮,用勺子刮去瓤 2、擦成细丝(没有擦菜板就用刀慢慢切成细丝) 3、锅烧热放油,入葱花煸出香味 4、入南瓜丝快速翻炒一分钟左右,放盐、一点白糖和鸡精调味出锅 2.香葱炒南瓜 原料:南瓜1只 调料:香葱、蒜末、橄榄油、盐 做法: 1、将南瓜去皮,切成片 2、油锅8成热后,将蒜末放入爆香 3、爆香后,将南瓜片放入,翻炒 4、在翻炒的同时,可以不时地往锅里加水,但不要太多 5、放入盐,炒匀 6、南瓜差不多软和绵了之后,就可以关火 7、撒入香葱,即可出锅"]


# Tokenize the input texts
batch_dict = new_tokenizer(input_texts, max_length=512, padding=True, truncation=True, return_tensors='pt')

outputs = model(**batch_dict)
embeddings = average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])

# normalize embeddings
embeddings = F.normalize(embeddings, p=2, dim=1)
scores = (embeddings[:2] @ embeddings[2:].T) * 100
print(scores.tolist())


[[90.00642395019531, 74.3570785522461], [73.54334259033203, 87.34696197509766]]


In [17]:
import torch.nn.functional as F

from torch import Tensor
from transformers import AutoTokenizer, AutoModel


def average_pool(last_hidden_states: Tensor,
                 attention_mask: Tensor) -> Tensor:
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]


# Each input text should start with "query: " or "passage: ", even for non-English texts.
# For tasks other than retrieval, you can simply use the "query: " prefix.
input_texts = ['query: how much protein should a female eat',
               'query: 南瓜的家常做法',
               "passage: As a general guideline, the CDC's average requirement of protein for women ages 19 to 70 is 46 grams per day. But, as you can see from this chart, you'll need to increase that if you're expecting or training for a marathon. Check out the chart below to see how much protein you should be eating each day.",
               "passage: 1.清炒南瓜丝 原料:嫩南瓜半个 调料:葱、盐、白糖、鸡精 做法: 1、南瓜用刀薄薄的削去表面一层皮,用勺子刮去瓤 2、擦成细丝(没有擦菜板就用刀慢慢切成细丝) 3、锅烧热放油,入葱花煸出香味 4、入南瓜丝快速翻炒一分钟左右,放盐、一点白糖和鸡精调味出锅 2.香葱炒南瓜 原料:南瓜1只 调料:香葱、蒜末、橄榄油、盐 做法: 1、将南瓜去皮,切成片 2、油锅8成热后,将蒜末放入爆香 3、爆香后,将南瓜片放入,翻炒 4、在翻炒的同时,可以不时地往锅里加水,但不要太多 5、放入盐,炒匀 6、南瓜差不多软和绵了之后,就可以关火 7、撒入香葱,即可出锅"]

tokenizer = AutoTokenizer.from_pretrained('intfloat/multilingual-e5-large')
model = AutoModel.from_pretrained('intfloat/multilingual-e5-large')

# Tokenize the input texts
batch_dict = tokenizer(input_texts, max_length=512, padding=True, truncation=True, return_tensors='pt')

outputs = model(**batch_dict)
embeddings = average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])

# normalize embeddings
embeddings = F.normalize(embeddings, p=2, dim=1)
scores = (embeddings[:2] @ embeddings[2:].T) * 100
print(scores.tolist())


[[90.81391143798828, 72.13390350341797], [70.53540802001953, 88.76109313964844]]
